# Exploration - postgreSQL - `pgvector` and `tsvector` 

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

from utils import execute, query, dense_search, sparse_search, rrf_results

c:\Users\de_ol\git\_git_d4g\14_EUFactForce\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Setup DB

Setup test db localy with a couple of records in a 'vectore_store' table, with:
- `id`: unique ID of the record
- `created_at`: creation datetime
- `content`: the text of the chunk
- `dense_vector`: embedding of the content
- `sparse_vector`: sparse bag-of-word representation of the vector 

In [2]:
# Enable pgvector
execute("CREATE EXTENSION IF NOT EXISTS vector;")

In [3]:
# Drop table if exists
execute("DROP TABLE IF EXISTS vector_store;")

In [4]:
# Create table
sql = f"""
CREATE TABLE IF NOT EXISTS vector_store (
  id uuid NOT NULL DEFAULT gen_random_uuid(),
  created_at timestamp with time zone DEFAULT now(),
  content text,
  dense_vector vector,
  sparse_vector tsvector
);
"""
execute(sql=sql)

In [5]:
# Generate fake chunks with content and embedding
sentences = [
    "This is a test",
    "This is another test",
    "This is a third test",
    "Test, test and test again",
    "Public health",
    "Testing public healthcare",
]
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(sentences).tolist()

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 156.09it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [6]:
# Insert rows (content, dense and sparse vectors)
data = [
    {"content": content, "dense_vector": embeddings[i]}
    for i, content in enumerate(sentences)
]

sql = f""" INSERT INTO vector_store (content, dense_vector, sparse_vector) VALUES
"""
for chunk in data:
    sql += f"('{chunk['content']}', '{chunk['dense_vector']}', to_tsvector('{chunk['content']}')),"
sql = sql[:-1] + ";"

execute(sql)

In [7]:
# Query table
chunks = query("SELECT * FROM vector_store;")
pd.DataFrame(chunks)

,id,created_at,content,dense_vector,sparse_vector
0,26d795b1-2a32-4ce9-ae58-b42fef2ab5bb,2026-02-27 08:30:47.924036+00:00,This is a test,"[0.03061251,0.01383137,-0.020843795,0.01632786...",'test':4
1,0307bbbf-999a-4569-b761-a0cea87f8bea,2026-02-27 08:30:47.924036+00:00,This is another test,"[0.029963816,-0.024454845,-0.03722259,0.011126...",'anoth':3 'test':4
2,dbba08d4-b1ea-4564-8f5b-98a3ba8ce739,2026-02-27 08:30:47.924036+00:00,This is a third test,"[0.02903016,-0.047188446,-0.061709963,0.030318...",'test':5 'third':4
3,89ec2241-25e1-4c75-b395-dfd628d9ddc5,2026-02-27 08:30:47.924036+00:00,"Test, test and test again","[-0.0037224826,-0.02382915,0.02301056,0.059218...","'test':1,2,4"
4,f1aeef17-30d2-442d-9ed7-2de10a87e77c,2026-02-27 08:30:47.924036+00:00,Public health,"[0.06317666,0.065679505,-0.018301733,0.0515413...",'health':2 'public':1
5,14eb8b0e-f4da-4de9-9908-5003dea86885,2026-02-27 08:30:47.924036+00:00,Testing public healthcare,"[-0.008998011,0.054530304,-0.06280593,-0.02360...",'healthcar':3 'public':2 'test':1


In [8]:
import json 
chunks_md = pd.DataFrame(chunks)
chunks_md['dense_vector'] = chunks_md['dense_vector'].apply(lambda x: json.dumps(x)[:25] + '...')
chunks_md.to_markdown()

'|    | id                                   | created_at                       | content                   | dense_vector                 | sparse_vector                     |\n|---:|:-------------------------------------|:---------------------------------|:--------------------------|:-----------------------------|:----------------------------------|\n|  0 | 26d795b1-2a32-4ce9-ae58-b42fef2ab5bb | 2026-02-27 08:30:47.924036+00:00 | This is a test            | "[0.03061251,0.01383137,-... | \'test\':4                          |\n|  1 | 0307bbbf-999a-4569-b761-a0cea87f8bea | 2026-02-27 08:30:47.924036+00:00 | This is another test      | "[0.029963816,-0.02445484... | \'anoth\':3 \'test\':4                |\n|  2 | dbba08d4-b1ea-4564-8f5b-98a3ba8ce739 | 2026-02-27 08:30:47.924036+00:00 | This is a third test      | "[0.02903016,-0.047188446... | \'test\':5 \'third\':4                |\n|  3 | 89ec2241-25e1-4c75-b395-dfd628d9ddc5 | 2026-02-27 08:30:47.924036+00:00 | Test, test and test aga

## 2. Search - Dense

Here we will encode an input text, and retrieve to top N vectors based on somilarity (by default, we will use cosine distance).

In [9]:
# Create input sentence and encode it
input = "Public Health tests"
model = SentenceTransformer("all-MiniLM-L6-v2")
input_vector = model.encode(input).tolist()

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 159.55it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
# Get top N simliar vectors in database using dense vector distance
dense_df = dense_search(input_vector, n=5)
dense_df

,id,content,distance,rank
0,14eb8b0e-f4da-4de9-9908-5003dea86885,Testing public healthcare,0.153110,1
1,f1aeef17-30d2-442d-9ed7-2de10a87e77c,Public health,0.253340,2
2,0307bbbf-999a-4569-b761-a0cea87f8bea,This is another test,0.558045,3
3,dbba08d4-b1ea-4564-8f5b-98a3ba8ce739,This is a third test,0.565464,4
4,26d795b1-2a32-4ce9-ae58-b42fef2ab5bb,This is a test,0.600432,5


## 3. Search - Sparse

Here we will transform the user input as a list of keywords, amd use `OR` conditions to retrieve most similar records (based of word occurence).

In [11]:
# Get top N simliar vectors in database using keywords and sparse vector rank
keywords = input.lower().replace(" ", "|")
sparse_df = sparse_search(keywords, n=5)
sparse_df

,id,content,similarity,rank
0,89ec2241-25e1-4c75-b395-dfd628d9ddc5,"Test, test and test again",0.3,1
1,f1aeef17-30d2-442d-9ed7-2de10a87e77c,Public health,0.2,2
2,14eb8b0e-f4da-4de9-9908-5003dea86885,Testing public healthcare,0.2,2
3,26d795b1-2a32-4ce9-ae58-b42fef2ab5bb,This is a test,0.1,4
4,0307bbbf-999a-4569-b761-a0cea87f8bea,This is another test,0.1,4


## 4. Search - Hybrid

Using both dense and sparse search, we will use [Reciprocal Rank Fusion](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) to combine both search into a single search. 

In [12]:
rrf_df = rrf_results(dense_df, sparse_df)
rrf_df

,id,score,content
1,14eb8b0e-f4da-4de9-9908-5003dea86885,0.032522,Testing public healthcare
5,f1aeef17-30d2-442d-9ed7-2de10a87e77c,0.032258,Public health
0,0307bbbf-999a-4569-b761-a0cea87f8bea,0.031498,This is another test
2,26d795b1-2a32-4ce9-ae58-b42fef2ab5bb,0.031010,This is a test
3,89ec2241-25e1-4c75-b395-dfd628d9ddc5,0.016393,"Test, test and test again"
